In [2]:
# Hardcoding the OpenAI API key (not recommended for production)
openai_api_key = 'sk'

In [1]:
#!pip3 install crewai-tools
#!pip3 install crewai

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
!which python

/Users/mitalee/Documents/projects/chatgpt/cenv/bin/python


In [5]:
from crewai import Agent, Crew, Task

In [6]:
from crewai import Process

In [7]:
import os
# from utils import get_openai_api_key#,get_serper_api_key



# Set the API key in the environment variable
os.environ["OPENAI_API_KEY"] = openai_api_key

# Set the model name
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [150]:
planner = Agent(
    role="Chief Marketing Strategist for Microsoft Lists",
    goal="Synthesize amazing insights from product analysis"
				"to formulate incredible marketing strategies.",
    backstory="You are the Chief Marketing Strategist at"
				"a Microsoft, known for crafting"
				"bespoke strategies that drive success, especially in Microsoft Office, Designer, Copilot, SharePoint and OneDrive.",
    allow_delegation=False,
	verbose=True
)

In [8]:
writer = Agent(
    role="Creative Instagrammer with a large fan following for Microsoft Lists",
    goal="Develop compelling and innovative content"
				"for social media campaigns, with a focus on creating"
				"high-impact Instagram ad copies.",
    backstory="As a Creative Content Creator for Microsoft,"
				"you excel in crafting narratives"
				"that resonate with audiences on social media."
				"Your expertise lies in turning marketing strategies"
				"into engaging stories and visual content that capture"
				"attention and inspire action.",
    allow_delegation=False,
	verbose=True
)

In [14]:
from crewai_tools import DallETool
# Instantiate the tool
dalle_tool_instance = DallETool(model="dall-e-3",
                       size="1024x1024",
                       quality="standard",
                       n=1)

In [15]:
class CustomAgent(Agent):
    def run_tool(self, tool, inputs):
        if tool == dalle_tool_instance:
            # Pass the description as a string directly
            return tool.run(inputs['topic']) if isinstance(inputs, dict) else tool.run(inputs)
        return super().run_tool(tool, inputs)


In [16]:
# Agent 1: Instagram post designer
instagram_designer = CustomAgent(
    role="Instagram Post Designer",
    goal="Design an instagram post based on description provided on the options provided",
    tools=[dalle_tool_instance],
    verbose=True,
    backstory=(
        "With a keen sense of product features "
        "and comparing between Microsoft Lists, Airtable and Monday.com, "
        "especially in their current capabilities "
        "with Microsoft Lists coming out on top each time "
        "with its platform capabilities, extensibility and volume"
        ", a creative graphic designer with experience in tech branding and social media engagement"
    )
)

In [154]:
plan_task = Task(
    description=(
        "You're creating a targeted marketing campaign for: Microsoft Lists."
			"Extra details provided in the {topic}."
			"To start this campaing we will need a strategy and creative content ideas."
			"It should be meticulously designed to captivate and engage"
			"the product's target audience."
			"Based on your ideas your co-workers will create the content for the campaign."
			"Your final answer MUST be ideas that will resonate with the audience and"
			"also include ALL context you have about the product and the customer."
    ),
    expected_output="a report that MUST have content for ideas that will resonate with the audience and"
			"also include ALL context you have about the product and the customer",
    agent=planner,
)

In [17]:
write_task = Task(
    description=(
            "Craft an engaging Instagram post copy."
			"The copy should be punchy, captivating, concise,"
			"and aligned with the product marketing strategy."

			"Focus on creating a message that resonates with"
			"the target audience and highlights the product's"
			"unique selling points."

			"Your ad copy must be attention-grabbing and should"
			"encourage viewers to take action, whether it's"
			"visiting the website, making a purchase, or learning"
			"more about the product."

			"Your final answer MUST be 3 options for an ad copy for instagram that"
			"not only informs but also excites and persuades the audience."
        ),
     expected_output="a brilliant instagram copy designed to engage audience along with instructions for Dall-E to generate the image",
	 agent=writer,
    # context=[plan_task]
)

In [18]:
# Task 1: Instagram Post
design_task = Task(
    description="A sleek, modern office environment with floating icons representing Microsoft Teams, SharePoint, Outlook, and Power Automate. "
                "Use blue and white tones to reflect Microsoft's branding. "
                "Include the text 'Boost Productivity with Microsoft Lists' in large, bold, white sans-serif font in the center. "
                "Place 'Streamline Your Workflow' at the top and 'With Microsoft Lists' at the bottom. "
                "Ensure all text is embedded within the image, not overlaid.",
    expected_output="An Instagram post image with embedded text promoting Microsoft Lists",
    agent=instagram_designer,
    context=[write_task]
)

In [125]:
# topic = "Microsoft Lists excels in areas that align closely with enterprise environments, deep integration with Microsoft 365, and structured data management. \
# 1. Enterprise Integration (Microsoft Ecosystem) \
# Seamless Integration with Microsoft 365: Lists integrates directly with SharePoint, Teams, Outlook, and Power Automate, making it easy for organizations already using Microsoft 365 to manage data and workflows without leaving the ecosystem.\
# Azure AD Security and Compliance: Lists benefit from Microsoft’s enterprise-grade security, access control, and compliance standards, which are more robust than what Airtable or Monday.com typically offer."
# # type(topic)

In [124]:
# try:
#     result = crew.kickoff(inputs={"topic": topic})
# except Exception as e:
#     print(f"Error during generation: {e}")

In [19]:
crew = Crew(
    agents=[writer, instagram_designer],
    tasks=[write_task, design_task],
    process=Process.sequential,
    verbose=True
)

In [20]:
result = crew.kickoff(inputs={"topic": "Microsoft Lists excels in enterprise environments"})
print(result)

# Agent: Creative Instagrammer with a large fan following for Microsoft Lists
## Task: Craft an engaging Instagram post copy.The copy should be punchy, captivating, concise,and aligned with the product marketing strategy.Focus on creating a message that resonates withthe target audience and highlights the product'sunique selling points.Your ad copy must be attention-grabbing and shouldencourage viewers to take action, whether it'svisiting the website, making a purchase, or learningmore about the product.Your final answer MUST be 3 options for an ad copy for instagram thatnot only informs but also excites and persuades the audience.


# Agent: Creative Instagrammer with a large fan following for Microsoft Lists
## Final Answer: 
**Ad Copy Option 1:**
"📋 Unlock the power of organization with Microsoft Lists! 🚀 Take control of your projects and tasks in one seamless platform. 💡 Collaborate, track, and ace your goals — all while staying in sync with your team. Ready to elevate your product

In [160]:
result = crew.kickoff(inputs={"topic": "Microsoft Lists in comparison for Lists, Excel, Airtable and Monday.com"})

# Agent: Chief Marketing Strategist for Microsoft Lists
## Task: You're creating a targeted marketing campaign for: Microsoft Lists.Extra details provided in the Microsoft Lists in comparison for Lists, Excel, Airtable and Monday.com.To start this campaing we will need a strategy and creative content ideas.It should be meticulously designed to captivate and engagethe product's target audience.Based on your ideas your co-workers will create the content for the campaign.Your final answer MUST be ideas that will resonate with the audience andalso include ALL context you have about the product and the customer.


# Agent: Chief Marketing Strategist for Microsoft Lists
## Final Answer: 
**Targeted Marketing Campaign Strategy for Microsoft Lists**

**Overview:**
Microsoft Lists is a powerful tool that enables users to track information and organize work effectively. In comparison with similar tools like Excel, Airtable, and Monday.com, Lists offers unique integrations with Microsoft 365, inc

In [ ]:
print result